In [8]:
import pandas as pd
import numpy as np
from scipy.stats import iqr
import matplotlib.pyplot as plt

In [9]:
data_path = 'weather.csv' 
data = pd.read_csv(data_path)

In [10]:
#Kita deteksi outlier pake IQR, sebenernya masih agak ragu apakah ada metode lain
#Tapi harusnya kalau cukup aman, karena yang dipake sebarang tengah data, haruse bisa kehindar dari nilai ekstrim
def deteksi_outlier_iqr_manual(data):
    outlier_indices = {}
    for kolom in data.select_dtypes(include=['number']).columns:
        Q1 = data[kolom].quantile(0.25)
        Q3 = data[kolom].quantile(0.75)
        IQR = Q3 - Q1
        batas_bawah = Q1 - 1.5 * IQR
        batas_atas = Q3 + 1.5 * IQR
        outlier_mask = (data[kolom] < batas_bawah) | (data[kolom] > batas_atas)
        outlier_indices[kolom] = data[kolom][outlier_mask].index.tolist()
    return outlier_indices

outlier_indices_manual = deteksi_outlier_iqr_manual(data)

print("Outlier yang terdeteksi dengan metode manual:")
for kolom, indices in outlier_indices_manual.items():
    if indices:
        print(f"{kolom}: {len(indices)} outliers")
    else:
        print(f"{kolom}: No outliers")

Outlier yang terdeteksi dengan metode manual:
Unnamed: 0: No outliers
Temperature: 13 outliers
Humidity: 11 outliers
Precipitation: 19 outliers
Wind_Speed: 10 outliers
Cloud_Coverage: 22 outliers
Pressure: 2 outliers
UV_Index: No outliers
Air_Quality: 23 outliers
Visibility: 21 outliers


In [13]:
def deteksi_outlier_iqr_library(data):
    outlier_indices = {}
    for kolom in data.select_dtypes(include=['number']).columns:
        kolom_data = data[kolom].dropna()
        IQR = iqr(kolom_data)
        Q1 = np.percentile(kolom_data, 25)
        Q3 = np.percentile(kolom_data, 75)
        batas_bawah = Q1 - 1.5 * IQR
        batas_atas = Q3 + 1.5 * IQR
        outlier_mask = (kolom_data < batas_bawah) | (kolom_data > batas_atas)
        outlier_indices[kolom] = kolom_data[outlier_mask].index.tolist()
    return outlier_indices

outlier_indices_library = deteksi_outlier_iqr_library(data)

print("\nOutlier yang terdeteksi dengan library:")
for kolom, indices in outlier_indices_library.items():
    if indices:
        print(f"{kolom}: {len(indices)} outliers")
    else:
        print(f"{kolom}: No outliers")


Outlier yang terdeteksi dengan library:
Unnamed: 0: No outliers
Temperature: 13 outliers
Humidity: 11 outliers
Precipitation: 19 outliers
Wind_Speed: 10 outliers
Cloud_Coverage: 22 outliers
Pressure: 2 outliers
UV_Index: No outliers
Air_Quality: 23 outliers
Visibility: 21 outliers


### Penanganan Outlier

1. **Penghapusan**: 
   - Menghapus nilai-nilai outlier dari dataset, terutama jika mereka disebabkan oleh kesalahan pengukuran atau pencatatan. : Cocok jika outlier adalah hasil dari kesalahan pengukuran atau jika mereka tidak mewakili populasi yang sedang dipelajari. Namun, ini bisa mengurangi ukuran sampel, yang mungkin tidak diinginkan jika dataset kecil.

2. **Imputasi**: 
   - Mengganti outlier dengan nilai yang lebih masuk akal, seperti median atau rata-rata dari data lainnya. Mengganti outlier dengan nilai median atau rata-rata dapat menjaga ukuran sampel, tetapi bisa memperkenalkan bias jika outlier sebenarnya adalah bagian dari distribusi data.

3. **Transformasi**: 
   - Melakukan transformasi data (seperti log atau z-score) untuk mengurangi dampak outlier. Jujur, sepertinya Transformasi seperti z-score atau log transformasi dapat mengurangi pengaruh outlier tanpa menghilangkan data. Ini bisa sangat efektif jika outlier adalah bagian dari distribusi yang diharapkan, tetapi mungkin akan berdampak besar pada analisis.

4. **Peningkatan Model**: 
   - Menggunakan model yang lebih robust terhadap outlier, seperti median regression atau menggunakan algoritma yang lebih tahan terhadap outlier. Menggunakan model yang lebih robust terhadap outlier, seperti median regression atau algoritma yang lebih tahan terhadap outlier, dapat sangat efektif. Namun, ini lebih kompleks dan cukup sulit.

### Pandangan Terhadap Outlier
Sepertinya metode yang paling cocok adalah imputasi outlier, untuk sementara. Dengan metode tersebut, ukuran sampel tetap terjaga dan mengurangi pengaruh outlier tanpa menghapus data yang mungkin penting untuk analisis di soal 5 dan 6. Pemilihan imputasi didasarkan oleh tingkat kesulitan implementasi yang tidak terlalu tinggi.


In [16]:
#Implementasi penanganan outlier

def imputasi_outlier(data, outlier_indices):
    for kolom, indices in outlier_indices.items():
        median = data[kolom].median()
        data.loc[indices, kolom] = median
    return data

outlier_indices_manual = deteksi_outlier_iqr_manual(data)

data_imputasi = imputasi_outlier(data.copy(), outlier_indices_manual)

print("\nStatistik deskriptif sebelum penanganan outlier:")
print(data.describe())

print("\nStatistik deskriptif setelah imputasi outlier:")
print(data_imputasi.describe())


Statistik deskriptif sebelum penanganan outlier:
        Unnamed: 0  Temperature     Humidity  Precipitation   Wind_Speed  \
count  2000.000000  2000.000000  2000.000000    2000.000000  2000.000000   
mean    999.500000    24.940225    70.110113       9.922872    14.929117   
std     577.494589     3.003306     9.807775       5.031691     4.972098   
min       0.000000    15.441317    40.991391      -9.480835    -4.063694   
25%     499.750000    22.919216    63.313635       6.695718    11.499557   
50%     999.500000    24.937000    69.871791       9.908852    14.890523   
75%    1499.250000    26.986035    76.744302      13.248399    18.242726   
max    1999.000000    35.060065   104.672989      26.851977    29.721840   

       Cloud_Coverage     Pressure     UV_Index  Air_Quality   Visibility  
count     2000.000000  2000.000000  2000.000000  2000.000000  2000.000000  
mean        49.856025  1000.691301     5.464730    65.757886    15.091752  
std         10.134826    57.329860   